In [2]:
!pip install pyspark findspark
import findspark
findspark.init()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=b99a517ddf781f0077f3590d40b50992cc3b288b02600b8ff3d4d59ae502f2fb
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [44]:
from pyspark.sql.functions import expr,col,isnan,when,count,regexp_replace,sum
from pyspark.sql.types import IntegerType,DoubleType,StringType
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("google_playstore_project").getOrCreate()

In [4]:
from google.colab import files
uploaded = files.upload()


Saving googleplaystore.csv to googleplaystore.csv


In [5]:
file_name = list(uploaded.keys())[0]

# Read the CSV file using PySpark
playstore_df = spark.read.csv(file_name, header=True, inferSchema=True)

# Display the data (use show() instead of display)
playstore_df.show(2)

+--------------------+--------------+------+-------+----+--------+----+-----+--------------+--------------------+----------------+-----------+------------+
|                 App|      Category|Rating|Reviews|Size|Installs|Type|Price|Content Rating|              Genres|    Last Updated|Current Ver| Android Ver|
+--------------------+--------------+------+-------+----+--------+----+-----+--------------+--------------------+----------------+-----------+------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 19M| 10,000+|Free|    0|      Everyone|        Art & Design| January 7, 2018|      1.0.0|4.0.3 and up|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967| 14M|500,000+|Free|    0|      Everyone|Art & Design;Pret...|January 15, 2018|      2.0.0|4.0.3 and up|
+--------------------+--------------+------+-------+----+--------+----+-----+--------------+--------------------+----------------+-----------+------------+
only showing top 2 rows



In [25]:
playstore_df.printSchema()

playstore_df.describe().show(2)

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)

+-------+-----+--------+------+-----------------+-----+-----------------+-----+-----+--------------+------+------------+-----------+-----------+
|summary|  App|Category|Rating|          Reviews| Size|         Installs| Type|Price|Content Rating|Genres|Last Updated|Current Ver|Android Ver|
+-------+-----+--------+------+-----------------+-----+-----------------+-----+-----+--------------+------+------------+-----------+-----------+
|  count|10841|   10841| 10841|            1

In [26]:
playstore_df1 = playstore_df.drop("Size","Content Rating","Last Updated","Current Ver","Android Ver")

In [27]:
playstore_df1.show(2)

+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|                 App|      Category|Rating|Reviews|Installs|Type|Price|              Genres|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 10,000+|Free|    0|        Art & Design|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967|500,000+|Free|    0|Art & Design;Pret...|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+
only showing top 2 rows



In [28]:
playstore_df1.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Genres: string (nullable = true)



In [55]:
playstore_df2 = playstore_df1.withColumn("Reviews",col("Reviews").cast(IntegerType()))\
.withColumn("Installs",regexp_replace(col("Installs"),"[^0-9]",""))\
.withColumn("Installs",col("Installs").cast(IntegerType()))\
.withColumn("Rating",col("Installs").cast(DoubleType()))\
.withColumn("Price",regexp_replace(col("Price"),"[$]",""))\
.withColumn("Price",(col("Price").cast(IntegerType())))
playstore_df2.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Reviews: integer (nullable = true)
 |-- Installs: integer (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: integer (nullable = true)
 |-- Genres: string (nullable = true)



In [56]:
#handling the missing data
playstore_df2.select([count(when(col(c).isNull(), c)).alias(c) for c in playstore_df2.columns]).show()
playstore_df2.count()


+---+--------+------+-------+--------+----+-----+------+
|App|Category|Rating|Reviews|Installs|Type|Price|Genres|
+---+--------+------+-------+--------+----+-----+------+
|  0|       0|     1|      3|       1|   0|    3|     0|
+---+--------+------+-------+--------+----+-----+------+



10841

In [57]:
#drop the rows that is missing
playstore_df2_clean=playstore_df2.dropna()
playstore_df2_clean.count()



10838

In [58]:
#filling specific values to the nan columns or missing columns
playstore_df2_filled=playstore_df2.fillna({"Reviews":0,"Rating":0,"Price":0,"installs":0})
playstore_df2_filled.count()
playstore_df2_filled.show(2)

+--------------------+--------------+--------+-------+--------+----+-----+--------------------+
|                 App|      Category|  Rating|Reviews|Installs|Type|Price|              Genres|
+--------------------+--------------+--------+-------+--------+----+-----+--------------------+
|Photo Editor & Ca...|ART_AND_DESIGN| 10000.0|    159|   10000|Free|    0|        Art & Design|
| Coloring book moana|ART_AND_DESIGN|500000.0|    967|  500000|Free|    0|Art & Design;Pret...|
+--------------------+--------------+--------+-------+--------+----+-----+--------------------+
only showing top 2 rows



In [59]:
playstore_df2_filled.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: double (nullable = false)
 |-- Reviews: integer (nullable = false)
 |-- Installs: integer (nullable = false)
 |-- Type: string (nullable = true)
 |-- Price: integer (nullable = false)
 |-- Genres: string (nullable = true)



In [60]:
from pyspark.sql import functions as F
gps_df = playstore_df2_filled.groupBy("App").agg(F.sum("Reviews").alias("Total_Reviews"))
gps_df = gps_df.orderBy(F.desc("Total_Reviews"))
gps_df.show(10)

+--------------------+-------------+
|                 App|Total_Reviews|
+--------------------+-------------+
|           Instagram|    266241989|
|  WhatsApp Messenger|    207348304|
|      Clash of Clans|    179558781|
|Messenger – Text ...|    169932272|
|      Subway Surfers|    166331958|
|    Candy Crush Saga|    156993136|
|            Facebook|    156286514|
|         8 Ball Pool|     99386198|
|        Clash Royale|     92530298|
|            Snapchat|     68045010|
+--------------------+-------------+
only showing top 10 rows



In [61]:
from pyspark.sql.functions import expr,col,isnan,when,count,regexp_replace,sum,desc
gps_df = playstore_df2_filled.groupBy("App").sum("Reviews")
gps_df = gps_df.orderBy(desc("sum(Reviews)"))
gps_df.show(10)

+--------------------+------------+
|                 App|sum(Reviews)|
+--------------------+------------+
|           Instagram|   266241989|
|  WhatsApp Messenger|   207348304|
|      Clash of Clans|   179558781|
|Messenger – Text ...|   169932272|
|      Subway Surfers|   166331958|
|    Candy Crush Saga|   156993136|
|            Facebook|   156286514|
|         8 Ball Pool|    99386198|
|        Clash Royale|    92530298|
|            Snapchat|    68045010|
+--------------------+------------+
only showing top 10 rows



In [62]:
gps_df = playstore_df2_filled.groupBy("App").agg(sum("Reviews").alias("Total_Reviews"))
gps_df = gps_df.orderBy(desc("Total_Reviews"))
gps_df.show(10)

+--------------------+-------------+
|                 App|Total_Reviews|
+--------------------+-------------+
|           Instagram|    266241989|
|  WhatsApp Messenger|    207348304|
|      Clash of Clans|    179558781|
|Messenger – Text ...|    169932272|
|      Subway Surfers|    166331958|
|    Candy Crush Saga|    156993136|
|            Facebook|    156286514|
|         8 Ball Pool|     99386198|
|        Clash Royale|     92530298|
|            Snapchat|     68045010|
+--------------------+-------------+
only showing top 10 rows



In [63]:
gps_df = playstore_df2_filled.groupBy("App").agg(sum("Installs").alias("Number_Apps"))
gps_df = gps_df.orderBy(desc("Number_Apps"))
gps_df.show(10)

+--------------------+-----------+
|                 App|Number_Apps|
+--------------------+-----------+
|      Subway Surfers| 6000000000|
|           Instagram| 4000000000|
|        Google Drive| 4000000000|
|            Hangouts| 4000000000|
|         Google News| 4000000000|
|       Google Photos| 4000000000|
|    Candy Crush Saga| 3500000000|
|Messenger – Text ...| 3000000000|
|Google Chrome: Fa...| 3000000000|
|Maps - Navigate &...| 3000000000|
+--------------------+-----------+
only showing top 10 rows



In [66]:
playstore_df2_filled.createOrReplaceTempView("apps")
result_df = spark.sql("SELECT * FROM apps LIMIT 5")
result_df.show()

+--------------------+--------------+---------+-------+--------+----+-----+--------------------+
|                 App|      Category|   Rating|Reviews|Installs|Type|Price|              Genres|
+--------------------+--------------+---------+-------+--------+----+-----+--------------------+
|Photo Editor & Ca...|ART_AND_DESIGN|  10000.0|    159|   10000|Free|    0|        Art & Design|
| Coloring book moana|ART_AND_DESIGN| 500000.0|    967|  500000|Free|    0|Art & Design;Pret...|
|U Launcher Lite –...|ART_AND_DESIGN|5000000.0|  87510| 5000000|Free|    0|        Art & Design|
|Sketch - Draw & P...|ART_AND_DESIGN|    5.0E7| 215644|50000000|Free|    0|        Art & Design|
|Pixel Draw - Numb...|ART_AND_DESIGN| 100000.0|    967|  100000|Free|    0|Art & Design;Crea...|
+--------------------+--------------+---------+-------+--------+----+-----+--------------------+



In [68]:
result_df1 = spark.sql("SELECT App, sum(Installs) FROM apps group by 1 order by 2 desc LIMIT 10")
result_df1.show()

+--------------------+-------------+
|                 App|sum(Installs)|
+--------------------+-------------+
|      Subway Surfers|   6000000000|
|           Instagram|   4000000000|
|            Hangouts|   4000000000|
|        Google Drive|   4000000000|
|         Google News|   4000000000|
|       Google Photos|   4000000000|
|    Candy Crush Saga|   3500000000|
|  WhatsApp Messenger|   3000000000|
|Messenger – Text ...|   3000000000|
|Google Chrome: Fa...|   3000000000|
+--------------------+-------------+



In [71]:
gps_df = playstore_df2_filled.groupBy("App","Type").agg(sum("Installs").alias("Apps_install_type"))
gps_df = gps_df.orderBy(desc("Apps_install_type"))
gps_df.show(10)

+------------------+----+-----------------+
|               App|Type|Apps_install_type|
+------------------+----+-----------------+
|    Subway Surfers|Free|       6000000000|
|         Instagram|Free|       4000000000|
|      Google Drive|Free|       4000000000|
|          Hangouts|Free|       4000000000|
|     Google Photos|Free|       4000000000|
|       Google News|Free|       4000000000|
|  Candy Crush Saga|Free|       3500000000|
|WhatsApp Messenger|Free|       3000000000|
|             Gmail|Free|       3000000000|
|      Temple Run 2|Free|       3000000000|
+------------------+----+-----------------+
only showing top 10 rows



In [70]:
result_df2 = spark.sql("SELECT App,Type, sum(Installs) FROM apps group by 1,2 order by 3 desc LIMIT 10")
result_df2.show()

+------------------+----+-------------+
|               App|Type|sum(Installs)|
+------------------+----+-------------+
|    Subway Surfers|Free|   6000000000|
|         Instagram|Free|   4000000000|
|      Google Drive|Free|   4000000000|
|          Hangouts|Free|   4000000000|
|     Google Photos|Free|   4000000000|
|       Google News|Free|   4000000000|
|  Candy Crush Saga|Free|   3500000000|
|WhatsApp Messenger|Free|   3000000000|
|             Gmail|Free|   3000000000|
|      Temple Run 2|Free|   3000000000|
+------------------+----+-------------+



In [72]:
gps_df = playstore_df2_filled.groupBy("Category").agg(sum("Installs").alias("Apps_install_Category"))
gps_df = gps_df.orderBy(desc("Apps_install_Category"))
gps_df.show(10)

+------------------+---------------------+
|          Category|Apps_install_Category|
+------------------+---------------------+
|              GAME|          35086024415|
|     COMMUNICATION|          32647276251|
|      PRODUCTIVITY|          14176091369|
|            SOCIAL|          14069867902|
|             TOOLS|          11452771915|
|            FAMILY|          10258263505|
|       PHOTOGRAPHY|          10088247655|
|NEWS_AND_MAGAZINES|           7496317760|
|  TRAVEL_AND_LOCAL|           6868887146|
|     VIDEO_PLAYERS|           6222002720|
+------------------+---------------------+
only showing top 10 rows



In [73]:
result_df2 = spark.sql("SELECT Category, sum(Installs) FROM apps group by 1 order by 2 desc LIMIT 10")
result_df2.show()

+------------------+-------------+
|          Category|sum(Installs)|
+------------------+-------------+
|              GAME|  35086024415|
|     COMMUNICATION|  32647276251|
|      PRODUCTIVITY|  14176091369|
|            SOCIAL|  14069867902|
|             TOOLS|  11452771915|
|            FAMILY|  10258263505|
|       PHOTOGRAPHY|  10088247655|
|NEWS_AND_MAGAZINES|   7496317760|
|  TRAVEL_AND_LOCAL|   6868887146|
|     VIDEO_PLAYERS|   6222002720|
+------------------+-------------+



In [76]:
gps_df = playstore_df2_filled.filter(col("Type") == 'Paid').groupBy("App").agg(sum("Price").alias("Apps_paid"))
gps_df = gps_df.orderBy(desc("Apps_paid"))
gps_df.show(10)

+--------------------+---------+
|                 App|Apps_paid|
+--------------------+---------+
|I'm Rich - Trump ...|      400|
|most expensive ap...|      399|
|           I am Rich|      399|
|  I AM RICH PRO PLUS|      399|
|  I am rich(premium)|      399|
|      I am Rich Plus|      399|
|I'm Rich/Eu sou R...|      399|
|   I Am Rich Premium|      399|
|       I Am Rich Pro|      399|
|          I am Rich!|      399|
+--------------------+---------+
only showing top 10 rows



In [75]:
# group by the Price and calculte sum of paid apps
result_df2 = spark.sql("SELECT App, sum(Price) FROM apps where Type = 'Paid' group by 1 order by 2 desc LIMIT 10")
result_df2.show()

+--------------------+----------+
|                 App|sum(Price)|
+--------------------+----------+
|I'm Rich - Trump ...|       400|
|most expensive ap...|       399|
|  I am rich(premium)|       399|
|   I Am Rich Premium|       399|
|      I am Rich Plus|       399|
|I'm Rich/Eu sou R...|       399|
|       I Am Rich Pro|       399|
|  I AM RICH PRO PLUS|       399|
|           I am Rich|       399|
|          I am Rich!|       399|
+--------------------+----------+

